In [1]:
import smokedduck
import pandas as pd

# Creating connect
con = smokedduck.prov_connect()

In [2]:
# Loading example data
customers_df = pd.DataFrame({'cid': [1, 2, 3], 'name': ['Alice', 'Bob', 'Ellen']})
con.execute('CREATE TABLE customers AS (SELECT * from customers_df)')
customers_df

,cid,name
0,1,Alice
1,2,Bob
2,3,Ellen


In [3]:
sales_df = pd.DataFrame({'oid': [101, 102, 103, 104, 105], 'cid': [1, 1, 2, 3, 3],'amount': [50, 30, 25, 40, 15]})
con.execute('CREATE TABLE sales AS (SELECT *  from sales_df)')
sales_df

,oid,cid,amount
0,101,1,50
1,102,1,30
2,103,2,25
3,104,3,40
4,105,3,15


In [4]:
# Executing base query
con.execute('SELECT cid, name, sum(amount) FROM customers JOIN sales USING (cid) GROUP BY cid, name ORDER BY cid', capture_lineage='lineage').df()


Enable Lineage Capture

Disable Lineage Capture


,cid,name,sum(amount)
0,1,Alice,80.0
1,2,Bob,25.0
2,3,Ellen,55.0


In [5]:
# Printing lineage that was captured from base query using different provenance models (e.g. lineage, why, polynomial, backward, forward).

In [7]:
# Lineage querying returns a column for each joined table in the base query. 
# The cell value is the index of the row that contributed to the out_index.
# If multiple rows from a table contribute to an output such as when there's an aggregation,
# multiple rows are returned by the lineage query (similar to Perm). Self-joins lead to multiple columns that correspond to that table.
con.lineage().df()

*****  


,sales,customers,out_index
0,1,0,0
1,2,1,1
2,4,2,2
3,0,0,0
4,3,2,2


In [8]:
# Why provenance produces a single row per out_index, creating a list of lists containing input indexes.
# Each internal list identifies one set of witnesses that contribute to the query. 
# Base query aggregation results in multiple internal lists.  
con.why().df()

*****  


,prov,out_index
0,"[[1, 0], [0, 0]]",0
1,"[[2, 1]]",1
2,"[[4, 2], [3, 2]]",2


In [9]:
# Polynomial returns how each out_index was created in the form of a polynomial. 
# Items multiplied together must co-occur, and those added together can either occur in order for the out_index to exist. 
# In general, joins produce multiplication and aggregations product addition.  
con.polynomial().df()

*****  


,prov,out_index
0,1*0+0*0,0
1,2*1,1
2,4*2+3*2,2


In [10]:
con.backward([0, 2]).df()

*****  


,sales,customers,out_index
0,1,0,0
1,4,2,2
2,0,0,0
3,3,2,2


In [11]:
con.backward([1, 2], 'polynomial').df()

*****  


,prov,out_index
0,2*1,1
1,4*2+3*2,2


In [12]:
con.forward('customers', [0, 1]).df()

*****  


,sales,customers,out_index
0,1,0,0
1,2,1,1
2,0,0,0


In [13]:
con.forward('sales', [2, 3]).df()

*****  


,sales,customers,out_index
0,2,1,1
1,3,2,2
